# AI Flappy Bird Analysis

## Flappy Bird no-GUI game engine

### Engine-specific imports and constants

In [ ]:
import random
import time

GAME_WIDTH = 500
GAME_HEIGHT = 700

### Bird class

In [ ]:
class Bird:
  def __init__(self, game):
    self.game = game

    self.x = self.game.GAME_WIDTH / 2 - 50
    self._y = self.game.GAME_HEIGHT / 2 - 70

    self.width = 50
    self.height = 40

    self.gravity = 600
    self.velocity = 0

  @property
  def y(self):
    return self._y;
  @y.setter
  def y(self, v):
    if (v > self.game.GAME_HEIGHT or v < 0 - self.height):
      self.game.gameIsOver = True
    else:
     self._y = v;

  @property
  def hitbox(self):
    return {
      'x': self.x,
      'y': self.y,
      'width': self.width,
      'height': self.height,
    }

  def update(self, dt):
    self.fall(dt)

  def reset(self):
    self.x = self.game.GAME_WIDTH / 2 - 50
    self._y = self.game.GAME_HEIGHT / 2 - 70

    self.velocity = 0

  def fall(self, dt):
    self.velocity += self.gravity * dt
    self.y += self.velocity * dt

  def flapWings(self):
    self.velocity = -300


### Pipes class

In [ ]:
class Pipes:
  def __init__(self, game):
    self.game = game

    self.startPoint = 500

    self.gap = 200

    self._x = self.startPoint
    self.topY = random.random() * (self.game.GAME_HEIGHT - self.gap)
    self.botY = self.topY + self.gap

    self.width = 75
    self.topHeight = self.topY
    self.botHeight = self.game.GAME_HEIGHT - self.botY

    self.velocity = 100

    self.passed = False

  @property
  def x(self):
    return self._x
  @x.setter
  def x(self, v):
    if (v + self.width < 0):
      self.game.removePipePair(self)
    else:
      self._x = v

  @property
  def topHitbox(self):
    return {
      'x': self.x,
      'y': 0,
      'width': self.width,
      'height': self.topHeight,
    }
  @property
  def botHitbox(self):
    return {
      'x': self.x,
      'y': self.botY,
      'width': self.width,
      'height': self.botHeight,
    }

  def update(self, dt):
    self.x -= self.velocity * dt

  def reset(self):
    self.x = self.startPoint
    self.topY = random.random() * (self.game.GAME_HEIGHT - self.gap)
    self.botY = self.topY + self.gap

    self.topHeight = self.topY
    self.botHeight = self.game.GAME_HEIGHT - self.botY


### Game class

In [ ]:
class Game:
  def __init__(self, GAME_WIDTH, GAME_HEIGHT):
    self.accumulator = 0.0

    self.pipesInterval = 3
    self.flappingInterval = 0.5
    self.deltaTime = 0
    self.lastTime = None
    self.pipesElapsed = 0
    self.flappingElapsed = 0

    self.GAME_WIDTH = GAME_WIDTH
    self.GAME_HEIGHT = GAME_HEIGHT

    self.bird = Bird(self)
    self.pipes = []

    self.gameIsOver = False

    self.points = 0

  def update(self, dt):
    for pipePair in self.pipes:
      if self.areColliding(self.bird.hitbox, pipePair.topHitbox) or \
      self.areColliding(self.bird.hitbox, pipePair.botHitbox):
        self.gameIsOver = True
        return

      if not(pipePair.passed) and self.bird.x > pipePair.x:
        self.points += 1
        pipePair.passed = True

      pipePair.update(dt)
    self.bird.update(dt)

  def gameLoop(self):
    currentTime = time.time()
    if self.lastTime is None:
      self.lastTime = currentTime
    dt = currentTime - self.lastTime
    self.deltaTime = dt
    self.lastTime = currentTime

    FIXED_DT = 0.01667

    self.accumulator += dt

    while self.accumulator >= FIXED_DT:
      if not(self.gameIsOver):
        self.update(FIXED_DT)
      self.accumulator -= FIXED_DT

    if self.gameIsOver:
      return
    else:
      self.pipesElapsed += dt

      if self.pipesElapsed >= self.pipesInterval:
        self.addPipePair()
        self.pipesElapsed = 0

  def reset(self):
    self.deltaTime = 0
    self.lastTime = None
    self.pipesElapsed = 0
    self.flappingElapsed = 0

    self.bird.reset()

    self.pipes = []

    self.points = 0

  def addPipePair(self):
    pipePair = Pipes(self)
    self.pipes.append(pipePair)
  def removePipePair(self, pipePair):
    if pipePair in self.pipes:
      self.pipes.remove(pipePair)

  def areColliding(self, bird, pipe):
    return bird['x'] < pipe['x'] + pipe['width'] and \
      bird['x'] + bird['width'] > pipe['x'] and \
      bird['y'] < pipe['y'] + pipe['height'] and \
      bird['y'] + bird['height'] > pipe['y'] - 30


## Game start!

In [ ]:
game = Game(GAME_WIDTH, GAME_HEIGHT)

while not game.gameIsOver:
  game.gameLoop()

  print(f"Bird: x={game.bird.x}, y={game.bird.y}")
  print(f"Pipe pair number: {len(game.pipes)}")
  if len(game.pipes) > 0:
    print(f"1st pipe pair: x={game.pipes[0].x}, topY={game.pipes[0].topY}, botY={game.pipes[0].botY}")
  print(f"Points: {game.points}")
  print(f"Game is over: {game.gameIsOver}")
  print(f"Last time: {game.lastTime}")
  print(f"Delta time: {game.deltaTime}")
  print(f"Accumulator: {game.accumulator}")
  print("-" * 20)

  time.sleep(0.01667)

Bird: x=200.0, y=280.0
Pipe pair number: 0
Points: 0
Game is over: False
Last time: 1767388026.7532108
Delta time: 0.0
Accumulator: 0.0
--------------------
Bird: x=200.0, y=280.16673334
Pipe pair number: 0
Points: 0
Game is over: False
Last time: 1767388026.7702212
Delta time: 0.01701045036315918
Accumulator: 0.0003404503631591789
--------------------
Bird: x=200.0, y=280.50020002
Pipe pair number: 0
Points: 0
Game is over: False
Last time: 1767388026.7871838
Delta time: 0.016962528228759766
Accumulator: 0.0006329785919189437
--------------------
Bird: x=200.0, y=281.00040004000005
Pipe pair number: 0
Points: 0
Game is over: False
Last time: 1767388026.8041728
Delta time: 0.01698899269104004
Accumulator: 0.000951971282958982
--------------------
Bird: x=200.0, y=281.6673334
Pipe pair number: 0
Points: 0
Game is over: False
Last time: 1767388026.8211272
Delta time: 0.016954421997070312
Accumulator: 0.0012363932800292937
--------------------
Bird: x=200.0, y=282.5010001
Pipe pair number

# NOTES
- rekreirat u pythonu js game bez GUIa jer je tu bolje ai radit i da iman sve za posli analizu
- proba san fps based na oba ali je bolje timebased da bude fair game